In [ ]:
import os
import json
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
from model import Detector
from PIL import Image
from utils import file_format_counter
from dataloader import SARDet
from torchsummary import summary

Data Exploration

In [ ]:
data_dir = "/home/hasanmog/CNN-VS-ViT/Datasets/SARDet"
train = os.listdir(os.path.join(data_dir + '/train'))
val = os.listdir(os.path.join(data_dir + '/val'))
len(train) , len(val)

In [ ]:
random.seed(50)
random.shuffle(train)
test = train[70000:]
train = train[:70000]
len(train) , len(test)

In [ ]:
index = 6000
img = Image.open(os.path.join(data_dir+'/train/'+train[index]))
print(img)
plt.imshow(img)

In [ ]:
png , jpg ,bmp = file_format_counter(train)         
print(f"train_set: png={png} , jpg={jpg} , bmp={bmp}")
png , jpg ,bmp = file_format_counter(val)         
print(f"val_set: png={png} , jpg={jpg} , bmp={bmp}")
png , jpg ,bmp = file_format_counter(test)         
print(f"test_set: png={png} , jpg={jpg} , bmp={bmp}")      

In [ ]:
print(train[0])

In [ ]:
train_json = os.path.join(data_dir+'/train.json')
val_json = os.path.join(data_dir+'/val.json')

with open(train_json , 'r') as file:
    train_anno = json.load(file)
    
train_anno.keys() , train_anno['images'][0] , train_anno['annotations'][0] , train_anno['categories']

In [ ]:
train_set = SARDet(data_dir= data_dir , imgs = train , mode = 'train')
val_set = SARDet(data_dir= data_dir , imgs = val , mode = 'val')
test_set = SARDet(data_dir= data_dir , imgs = test , mode = 'test')
len(train_set) , len(val_set) , len(test_set)

In [ ]:
sample = val_set[4]
sample

In [ ]:
image = sample['image_tensor'].numpy()
image = np.transpose(image , (1 , 2 , 0))
print("image size : " , image.shape)

fig , ax = plt.subplots(1)
ax.imshow(image , cmap='viridis')

for i in range(len(sample['bboxes'])):
    box = sample['bboxes'][i]
    rect = patches.Rectangle((box[0], box[1]), box[2], box[3], linewidth=3, edgecolor='r', facecolor='none')
    plt.gca().add_patch(rect)
    
plt.show()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = Detector().to(device)
summary(model , input_size=(3,256 , 256))

In [17]:
input_array = np.random.randint(1, 3, (800, 800, 3))  # Generate a random 800x800 image with 3 channels (RGB)
input_tensor = torch.from_numpy(input_array).float()  # Convert to float tensor

# Permute dimensions to match [N, C, H, W] format expected by PyTorch convolutional layers
input_tensor = input_tensor.permute(2, 0, 1).unsqueeze(0)  # Add a batch dimension

# Assuming the model expects floating point data normalized to [0, 1]
input_tensor /= 3  # Simple scaling by max value of np.random.randint range to bring values between 0 and 1
input_tensor = input_tensor.to(device)
# Pass the input tensor through the model
outputs = model(input_tensor)  # Ensure model is in evaluation mode if not training: model.eval()


In [ ]:
outputs.shape